In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import io
import os
import time
import re
import json
import sys

# 家樂福線上購物網爬蟲
先爬一頁，若要翻頁可以觀察換頁後網址的規律，用迴圈方式寫

In [3]:
# 商品分類清單獲得
BASE_URL = 'https://online.carrefour.com.tw/zh/homepage'

rsp = requests.get(BASE_URL)
soup = BeautifulSoup(rsp.text, 'html.parser')
startQuery = False
for top1 in soup.findAll('div',{'class':'top1 left-item'}):
    for a in top1.findAll('a'):
        h3 = a.find('h3')
        span = a.find('span')
        if h3 != None:
            cls_1 = h3.text.replace('．','')
            path_1 = a.get('href')
        if span != None:
            cls_2 = span.text.replace('．','')
            path_2 = a.get('href')
            print(cls_1 , cls_2 , path_2)

蔬菜水果禮盒 各式蔬菜 /zh/%E7%94%9F%E9%AE%AE%E9%A3%9F%E5%93%81/%E8%94%AC%E8%8F%9C%E6%B0%B4%E6%9E%9C%E7%A6%AE%E7%9B%92/%E5%90%84%E5%BC%8F%E8%94%AC%E8%8F%9C
蔬菜水果禮盒 季節水果 /zh/%E7%94%9F%E9%AE%AE%E9%A3%9F%E5%93%81/%E8%94%AC%E8%8F%9C%E6%B0%B4%E6%9E%9C%E7%A6%AE%E7%9B%92/%E5%AD%A3%E7%AF%80%E6%B0%B4%E6%9E%9C
蔬菜水果禮盒 佐料 /zh/%E7%94%9F%E9%AE%AE%E9%A3%9F%E5%93%81/%E8%94%AC%E8%8F%9C%E6%B0%B4%E6%9E%9C%E7%A6%AE%E7%9B%92/%E4%BD%90%E6%96%99
蔬菜水果禮盒 菇菌類 /zh/%E7%94%9F%E9%AE%AE%E9%A3%9F%E5%93%81/%E8%94%AC%E8%8F%9C%E6%B0%B4%E6%9E%9C%E7%A6%AE%E7%9B%92/%E8%8F%87%E8%8F%8C%E9%A1%9E
蔬菜水果禮盒 禮盒 /zh/%E7%94%9F%E9%AE%AE%E9%A3%9F%E5%93%81/%E8%94%AC%E8%8F%9C%E6%B0%B4%E6%9E%9C%E7%A6%AE%E7%9B%92/%E7%A6%AE%E7%9B%92
人氣熟食 涼拌小菜冷盤 /zh/%E7%94%9F%E9%AE%AE%E9%A3%9F%E5%93%81/%E4%BA%BA%E6%B0%A3%E7%86%9F%E9%A3%9F/%E6%B6%BC%E6%8B%8C%E5%B0%8F%E8%8F%9C%E5%86%B7%E7%9B%A4
人氣熟食 素食 /zh/%E7%94%9F%E9%AE%AE%E9%A3%9F%E5%93%81/%E4%BA%BA%E6%B0%A3%E7%86%9F%E9%A3%9F/%E7%B4%A0%E9%A3%9F
人氣熟食 加工製品 /zh/%E7%94%9F%E9%AE%AE%E9%A3%9F%E5%93%81/%E4%BA%BA%E6%B0%A3%E7%86

In [4]:
# 蔬菜水果禮盒全品項爬
#https://online.carrefour.com.tw/zh/生鮮食品/水果蔬菜禮盒/各式蔬菜?start=24#
import time
tStart = time.time()#計時開始

URL = "https://online.carrefour.com.tw/zh/%E7%94%9F%E9%AE%AE%E9%A3%9F%E5%93%81/%E8%94%AC%E8%8F%9C%E6%B0%B4%E6%9E%9C%E7%A6%AE%E7%9B%92/%E5%90%84%E5%BC%8F%E8%94%AC%E8%8F%9C"
carr_Item = []
carr_Price = []
carr_URL = []

headers = {'user-agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36'}
rsp = requests.get(URL, headers = headers) 
rsp.encoding = 'utf8'
soup = BeautifulSoup(rsp.text, 'html.parser')

#第一頁
for item in soup.findAll(name='div', attrs={'class':'desc-operation-wrapper'}):
    for a in item.findAll('a'):
        item_url = a.get('href')
        item_name = a.get('title')
    for em in item.findAll('em'):
        item_price = em.getText()
        carr_Item.append(item_name)
        carr_Price.append(item_price)
        carr_URL.append(item_url)
        print(f"品項網址：{item_url}\n品項名稱：{item_name}\n價格：{item_price}\n")
        
#計時結束
tEnd = time.time()
#列印結果
tRun = tEnd - tStart
print('run time:%d'%tRun)

品項網址：/zh/2202010700101.html
品項名稱：進口青花菜2入(每盒約400克)
價格：$99

品項網址：/zh/%E7%94%A2%E9%8A%B7%E5%B1%A5%E6%AD%B7%E8%BE%B2%E7%94%A2%E5%93%81/2201014500101.html
品項名稱：履歷特選紅寶石紅蘿蔔(每袋約300克)
價格：$28

品項網址：/zh/%E5%AE%B6%E6%A8%82%E7%A6%8F/2202123600101.html
品項名稱：家樂福履歷彩色甜椒(每盒約300g)
價格：$59

品項網址：/zh/2202110700101.html
品項名稱：玉米筍(每盒約100g)
價格：$49

品項網址：/zh/%E7%94%A2%E9%8A%B7%E5%B1%A5%E6%AD%B7%E8%BE%B2%E7%94%A2%E5%93%81/2201101500101.html
品項名稱：履歷帶殼筊白筍(每袋約600克)
價格：$109

品項網址：/zh/%E6%9C%89%E6%A9%9F%E8%BE%B2%E7%94%A2%E5%93%81/2204021500501.html
品項名稱：有機珊瑚菇(每盒約100克)
價格：$45

品項網址：/zh/%E7%94%A2%E9%8A%B7%E5%B1%A5%E6%AD%B7%E8%BE%B2%E7%94%A2%E5%93%81/2200031100101.html
品項名稱：家樂福履歷小白菜(每袋約250克±10%)
價格：$36

品項網址：/zh/%E5%AE%B6%E6%A8%82%E7%A6%8F/2204029500101.html
品項名稱：家樂福履歷香菇(每盒約400克)
價格：$99

品項網址：/zh/%E5%AE%B6%E6%A8%82%E7%A6%8F/2204030000501.html
品項名稱：家樂福有機白精靈200g
價格：$39

品項網址：/zh/2202104100101.html
品項名稱：牛蕃茄(每盒約600克)
價格：$105

品項網址：/zh/2201010200101.html
品項名稱：西芹(約600公克)
價格：$79

品項網址：/zh/%E5%AE%B6%E6%A8%82%E7%A6%8F/22021149001

In [5]:
print(carr_Item)

['進口青花菜2入(每盒約400克)', '履歷特選紅寶石紅蘿蔔(每袋約300克)', '家樂福履歷彩色甜椒(每盒約300g)', '玉米筍(每盒約100g)', '履歷帶殼筊白筍(每袋約600克)', '有機珊瑚菇(每盒約100克)', '家樂福履歷小白菜(每袋約250克±10%)', '家樂福履歷香菇(每盒約400克)', '家樂福有機白精靈200g', '牛蕃茄(每盒約600克)', '西芹(約600公克)', '家樂福嚴選台灣有機紅鬚玉米筍(每袋約300公克±10%)', '家樂福嚴選木瓜型南瓜(每粒約800克 ±10%)', '胡瓜-條', '家樂福嚴選黃肉地瓜(每包約600克±10%)', '家樂福履歷空心菜(每包約250公克±10%)', '進口洋蔥每袋約1kg', '【安心價】家樂福有機杏鮑菇(每包約1公斤)', '履歷猴絨香菇(每盒約300克)', '台灣履歷烏殼綠竹筍(每支約800克)', '台灣有機胖胖菇(每包約450g)', '台灣有機五福菇(每盒約600克)', '【安心價】日本北海道山藥(L) (每條約600克)', '美國三色洋蔥 900克']


In [6]:
c={"Name" : carr_Item,
   "Price" : carr_Price,
   "URL" : carr_URL}
df = pd.DataFrame(c)
df['Date'] = time.strftime('%Y-%m-%d' , time.localtime())
df = pd.DataFrame(df,columns=['Date','Name', 'Price', 'URL'])
print(len(df))

24


In [7]:
df

,Date,Name,Price,URL
0,2021-07-20,進口青花菜2入(每盒約400克),$99,/zh/2202010700101.html
1,2021-07-20,履歷特選紅寶石紅蘿蔔(每袋約300克),$28,/zh/%E7%94%A2%E9%8A%B7%E5%B1%A5%E6%AD%B7%E8%BE...
2,2021-07-20,家樂福履歷彩色甜椒(每盒約300g),$59,/zh/%E5%AE%B6%E6%A8%82%E7%A6%8F/2202123600101....
3,2021-07-20,玉米筍(每盒約100g),$49,/zh/2202110700101.html
4,2021-07-20,履歷帶殼筊白筍(每袋約600克),$109,/zh/%E7%94%A2%E9%8A%B7%E5%B1%A5%E6%AD%B7%E8%BE...
5,2021-07-20,有機珊瑚菇(每盒約100克),$45,/zh/%E6%9C%89%E6%A9%9F%E8%BE%B2%E7%94%A2%E5%93...
6,2021-07-20,家樂福履歷小白菜(每袋約250克±10%),$36,/zh/%E7%94%A2%E9%8A%B7%E5%B1%A5%E6%AD%B7%E8%BE...
7,2021-07-20,家樂福履歷香菇(每盒約400克),$99,/zh/%E5%AE%B6%E6%A8%82%E7%A6%8F/2204029500101....
8,2021-07-20,家樂福有機白精靈200g,$39,/zh/%E5%AE%B6%E6%A8%82%E7%A6%8F/2204030000501....
9,2021-07-20,牛蕃茄(每盒約600克),$105,/zh/2202104100101.html


In [8]:
# 檢查df有沒有高麗菜
df.loc[df['Name'].str.contains('高麗菜')]

,Date,Name,Price,URL


In [43]:
# 第一次存新的檔案
df.to_csv('./carrefour_new.csv', index = False)

In [9]:
#之前存過一個CSV檔，想把之後爬的直接接在之前資料的後面
df.to_csv('./carrefour.csv', index=False, mode='a', header=False)

In [10]:
# look up updates
k = pd.read_csv('./carrefour.csv')
k

,Date,Name,Price,URL
0,2021-07-19,進口青花菜2入(每盒約400克),$99,/zh/2202010700101.html
1,2021-07-19,玉米筍(每盒約100g),$49,/zh/2202110700101.html
2,2021-07-19,家樂福履歷彩色甜椒(每盒約300g),$59,/zh/%E5%AE%B6%E6%A8%82%E7%A6%8F/2202123600101....
3,2021-07-19,履歷帶殼筊白筍(每袋約600克),$109,/zh/%E7%94%A2%E9%8A%B7%E5%B1%A5%E6%AD%B7%E8%BE...
4,2021-07-19,牛蕃茄(每盒約600克),$105,/zh/2202104100101.html
5,2021-07-19,家樂福履歷香菇(每盒約400克),$99,/zh/%E5%AE%B6%E6%A8%82%E7%A6%8F/2204029500101....
6,2021-07-19,履歷特選紅寶石紅蘿蔔(每袋約300克),$28,/zh/%E7%94%A2%E9%8A%B7%E5%B1%A5%E6%AD%B7%E8%BE...
7,2021-07-19,家樂福嚴選台灣有機紅鬚玉米筍(每袋約300公克±10%),$45,/zh/%E5%AE%B6%E6%A8%82%E7%A6%8F/2202114900101....
8,2021-07-19,西芹(約600公克),$79,/zh/2201010200101.html
9,2021-07-19,家樂福履歷小白菜(每袋約250克±10%),$36,/zh/%E7%94%A2%E9%8A%B7%E5%B1%A5%E6%AD%B7%E8%BE...


# 熊媽媽買菜網線上爬蟲

In [2]:
# 商品分類清單獲得
BASE_URL = 'https://www.happy-shopping.tw/'
index = 'index.php'
rsp = requests.get(BASE_URL+index)
soup = BeautifulSoup(rsp.text, 'html.parser')
startQuery = False
# print(soup.prettify())
for menu in soup.findAll('ul',{'class':'menu_nav_row2'}):
    for li in menu.findAll('li'):
        a = li.find('a')
        cls = a.getText()
        if a != None:
            path = a.get('href')
            print(cls , path)

蔬菜 second_class.php?fcid=1
水果 second_class.php?fcid=236
肉品 second_class.php?fcid=2
海鮮 second_class.php?fcid=3
特色熟食 second_class.php?fcid=4
快速美食 second_class.php?fcid=322
油．調味 second_class.php?fcid=5
南北貨 second_class.php?fcid=6
蛋品．米麵 second_class.php?fcid=241
料理食材包 second_class.php?fcid=11
胡媽媽灶腳 search_list.php?qry=胡媽媽灶腳
五星飯店美食 second_class.php?fcid=398


# 此方法看不到品項名稱
BASE_URL = 'https://www.happy-shopping.tw/'
path = 'second_class.php?fcid=236'
index = '/index.php'
rsp = requests.get(BASE_URL+path+index)
soup = BeautifulSoup(rsp.text, 'html.parser')
startQuery = False
print(soup.prettify())

In [9]:
# 爬蔬菜和水果
import time
tStart = time.time()#計時開始
BASE_URL = 'https://www.happy-shopping.tw/'
path_lst=['second_class.php?fcid=1', 'second_class.php?fcid=236']
class_lst = ['蔬菜', '水果']
Class = []
Item = []
Price = []
Img = []
URL = []


for i in range(len(path_lst)):
    print(class_lst[i])
    cls = class_lst[i]
    path = path_lst[i]
    chromedriver = '/usr/local/bin/chromedriver' #要用變數指定chromedriver的放置路徑
    bearmama = webdriver.Chrome(chromedriver)
    bearmama.get(BASE_URL+path)
    #將滾動條移動到到頁面的底部
    for i in range(1,15):
        js="var q=document.documentElement.scrollTop=100000"  
        bearmama.execute_script(js)  
        time.sleep(2)  
    soup = BeautifulSoup(bearmama.page_source, 'html.parser')
    bearmama.close()
    
    for img in soup.findAll('div',{'class':'item-img'}):
        item_img = img.find('img').get('src')
    for item in soup.findAll('div',{'class':'item-content'}):
        a = item.find('a')
        item_name = a.getText()
        if a != None:
            item_path = a.get('href')
            item_img = a.get('src')
            for span in item.findAll('span',{'class':'discount'}):
                price_d = span.getText()
                Item.append(item_name)
                Price.append(price_d)
                Img.append(item_img)
                URL.append(item_path)
                Class.append(cls)

#計時結束
tEnd = time.time()
#列印結果
tRun = tEnd - tStart
print('run time:%d'%tRun)

蔬菜
水果
run time:66


In [10]:
c={"Class" : Class,
   "Item" : Item,
   "Price" : Price,
   "URL" : URL,
   "Img" : Img}
df = pd.DataFrame(c)
df['Date'] = time.strftime('%Y-%m-%d' , time.localtime())
df = pd.DataFrame(df,columns=['Date','Class','Item', 'Price', 'URL','Img'])
print(len(df))

592


In [11]:
df

,Date,Class,Item,Price,URL,Img
0,2021-08-29,蔬菜,熊媽媽配送 冬瓜(400公克±10%),$20,product.php?fcid=1&scid=&pid=8589,None
1,2021-08-29,蔬菜,熊媽媽配送 酸白菜300公克(半斤)±10%,$108,product.php?fcid=1&scid=&pid=3797,None
2,2021-08-29,蔬菜,熊媽媽配送 梅子客家酸菜300公克(半斤)±10%,$67,product.php?fcid=1&scid=&pid=3156,None
3,2021-08-29,蔬菜,熊媽媽配送 青蔥150g±10%,$45,product.php?fcid=1&scid=&pid=3350,None
4,2021-08-29,蔬菜,熊媽媽配送 青木瓜1粒(800公克±10%),$113,product.php?fcid=1&scid=&pid=3397,None
...,...,...,...,...,...,...
587,2021-08-29,水果,廠商直送 【旬鮮屋】美國華盛頓9.5ROW櫻桃盛夏限定2kg (2公斤±10%)-熊,$1150,product.php?fcid=236&scid=&pid=2107170002,None
588,2021-08-29,水果,廠商直送 【禾鴻】產地直送枋山愛文芒果10斤18-24顆x2箱(7/20~7/26出貨),$1129,product.php?fcid=236&scid=&pid=2107100056,None
589,2021-08-29,水果,廠商直送 【禾鴻】產地直送枋山愛文芒果6-8顆x4箱(7/13~7/19出貨),$2756,product.php?fcid=236&scid=&pid=2107100057,None
590,2021-08-29,水果,廠商直送 【禾鴻】產地直送枋山愛文芒果6-8顆x1箱(7/13~7/19出貨),$799,product.php?fcid=236&scid=&pid=2107100068,None


In [19]:
# 檢查df有沒有高麗菜
df.loc[df['Item'].str.contains('高麗')]

,Date,Class,Item,Price,URL
82,2021-07-20,蔬菜,熊媽媽配送 紫高麗(1000g-1200g),$259,product.php?fcid=1&scid=&pid=1911030001
109,2021-07-20,蔬菜,熊媽媽配送 梨山高麗菜一顆(1000g-1200g),$137,product.php?fcid=1&scid=&pid=2009280016
113,2021-07-20,蔬菜,熊媽媽配送 梨山高麗菜一顆(1300g-1800g),$190,product.php?fcid=1&scid=&pid=2009280017


In [53]:
# 第一次存新的檔案
df.to_csv('./bearmama_new.csv', index = False)

In [22]:
#之前存過一個CSV檔，想把之後爬的直接接在之前資料的後面
df.to_csv('./bearmama.csv', index=False, mode='a', header=False)

In [23]:
# look up updates
b = pd.read_csv('./bearmama.csv')
b

,Date,Class,Item,Price,URL
0,2021-07-19,蔬菜,熊媽媽配送 牛蒡(400公克±10%),$99,product.php?fcid=1&scid=&pid=9131
1,2021-07-19,蔬菜,熊媽媽配送 芹菜200公克±10%,$75,product.php?fcid=1&scid=&pid=204
2,2021-07-19,蔬菜,熊媽媽配送 冬瓜(400公克±10%),$30,product.php?fcid=1&scid=&pid=8589
3,2021-07-19,蔬菜,熊媽媽配送 人蔘山藥600公克(1斤)±10%,$173,product.php?fcid=1&scid=&pid=7314
4,2021-07-19,蔬菜,熊媽媽配送 產銷履歷空心菜250公克±10%,$40,product.php?fcid=1&scid=&pid=10926
...,...,...,...,...,...
967,2021-07-20,水果,廠商直送 【禾鴻】產地直送枋山愛文芒果6-8顆x1箱(7/13~7/19出貨),$799,product.php?fcid=236&scid=&pid=2107100068
968,2021-07-20,水果,廠商直送 【禾鴻】產地直送枋山愛文芒果6-8顆x1箱(7/20~7/26出貨),$799,product.php?fcid=236&scid=&pid=2107100069
969,2021-07-20,水果,廠商直送 【禾鴻】產地直送枋山愛文芒果6-8顆x4箱(7/20~7/26出貨),$2756,product.php?fcid=236&scid=&pid=2107100070
970,2021-07-20,水果,廠商直送 【禾鴻】產地直送枋山愛文芒果6-8顆x2箱(7/13~7/19出貨),$1458,product.php?fcid=236&scid=&pid=2107100073
